# Churn Modelling

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

In [44]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


## EDA

In [46]:
# sns.pairplot(data=data,hue='Exited')

## Data Preprocessing

In [47]:
categorical_columns = ['Geography', 'Gender']

In [48]:
# One hot encoding
from sklearn.preprocessing import OneHotEncoder
oneHotEncoder = OneHotEncoder(sparse_output=False, drop='first')
one_hot_encoded = oneHotEncoder.fit_transform(data[categorical_columns])
one_hot_encoded_columns = oneHotEncoder.get_feature_names_out(categorical_columns)
one_hot_encoded_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoded_columns)
one_hot_encoded_df.head()

,Geography_Germany,Geography_Spain,Gender_Male
0,0.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,1.0,0.0


In [49]:
# Merging dataframes
data=pd.concat([data,one_hot_encoded_df],axis=1)
data.drop(categorical_columns, axis=1)
data.head() 

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,0.0,0.0,0.0
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0,0.0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,0.0,0.0,0.0
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0.0,0.0,0.0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0,0.0


In [50]:
X = data[['CreditScore', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Geography_Germany', 'Geography_Spain',
       'Gender_Male']]
y = data['Exited']

In [51]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [52]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [75]:
# Saving the scaler and encoder
import pickle

with open('onehotencoder.pkl', 'wb') as file:
    pickle.dump(oneHotEncoder, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## Tensorflow and Keras

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [55]:
# Dropout Regularization to overcome overfitting
model = Sequential()
model.add(Dense(6,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [56]:
# Compile model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [57]:
# Setup tensorboard and early stopping
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorBoard_callback = TensorBoard(log_dir, histogram_freq = 1)
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [58]:
model.fit(X_train,y_train.values,validation_data=(X_test, y_test.values),epochs=20,batch_size=10,callbacks=[tensorBoard_callback, early_stopping_callback])

Epoch 1/20


800/800 [==============================] - 6s 4ms/step - loss: 0.5339 - accuracy: 0.7959 - val_loss: 0.4794 - val_accuracy: 0.7965
Epoch 2/20
800/800 [==============================] - 2s 3ms/step - loss: 0.4705 - accuracy: 0.7962 - val_loss: 0.4492 - val_accuracy: 0.7965
Epoch 3/20
800/800 [==============================] - 2s 2ms/step - loss: 0.4488 - accuracy: 0.7962 - val_loss: 0.4367 - val_accuracy: 0.7965
Epoch 4/20
800/800 [==============================] - 2s 2ms/step - loss: 0.4432 - accuracy: 0.7986 - val_loss: 0.4318 - val_accuracy: 0.8050
Epoch 5/20
800/800 [==============================] - 2s 3ms/step - loss: 0.4379 - accuracy: 0.8141 - val_loss: 0.4269 - val_accuracy: 0.8140
Epoch 6/20
800/800 [==============================] - 2s 2ms/step - loss: 0.4329 - accuracy: 0.8148 - val_loss: 0.4226 - val_accuracy: 0.8175
Epoch 7/20
800/800 [==============================] - 2s 2ms/step - loss: 0.4291 - accuracy: 0.8167 - val_loss: 0.4190 - val_accuracy: 0.8135
Epoch 8/20
800/80

In [59]:
# Save model
model.save('model.h5')

## Tensorboard

In [60]:
# Load tensorboard extension
# %load_ext tensorboard

In [61]:
# %tensorboard --logdir logs/fit

## Model Evaluation

In [62]:
test_loss,test_acc = model.evaluate(X_test,y_test)
print("Test Loss:",test_loss,"\nTest Accuracy:",test_acc)

63/63 [==============================] - 1s 7ms/step - loss: 0.3839 - accuracy: 0.8250
Test Loss: 0.38392210006713867 
Test Accuracy: 0.824999988079071


In [63]:
predictions = model.predict(X_test)

63/63 [==============================] - 0s 2ms/step


In [64]:
predictions = (predictions>0.5)

In [65]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,predictions))

[[1543   50]
 [ 300  107]]


## Predicting Single Data

In [66]:
dict = {'CreditScore':600, 'Age':40, 'Tenure':3,
       'Balance':60000, 'NumOfProducts':2, 'HasCrCard':1, 'IsActiveMember':1,
       'EstimatedSalary':50000, 'Geography_Germany':0, 'Geography_Spain':0,
       'Gender_Male':1}
test_data = pd.DataFrame([dict])

In [67]:
test_data = scaler.transform(test_data)

In [68]:
pred = model.predict(test_data)

1/1 [==============================] - 0s 22ms/step


In [69]:
pred

array([[0.09474226]], dtype=float32)

In [70]:
pred = (pred>0.5)
pred[0][0]

False

## Hyperparameter Tuning

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [71]:
def create_model(neurons=32, layers=1):
    model=Sequential()
    
    # Input layer
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[0],)))

    # Hidden layers
    for _ in range(layers):
        model.add(Dense(neurons, activation='relu'))
    
    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [72]:
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(layers=1, neurons=32, build_fn=create_model, verbose=1)

In [73]:
# Grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [74]:
# from sklearn.model_selection import GridSearchCV

# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=5, cv=3, verbose=1)
# grid_result = grid.fit(X_train, y_train)

# print(f'Best: {grid_result.best_score_} using: {grid_result.best_params_}')